## **UNet Training and Testing**
Used in this repository for super-resolution image noise detection, but can be trained for other purposes.

**Code:** https://github.com/zhixuhao/unet

**Paper:** Ronneberger O., Fischer P., Brox T. (2015) U-Net: Convolutional Networks for Biomedical Image Segmentation. In: Navab N., Hornegger J., Wells W., Frangi A. (eds) Medical Image Computing and Computer-Assisted Intervention – MICCAI 2015. MICCAI 2015. Lecture Notes in Computer Science, vol 9351. Springer, Cham. https://doi.org/10.1007/978-3-319-24574-4_28

---

**Begin:**  The first step, is to gather your data into two directories (folders).  One for the original images to be processed and one for data labels.

Additionally, if you are testing only and do not have labels, that is fine.  If you are conducting training, labels are required.  Annotation files are not required.

---
NOTE: To activate GPU, go to Runtime (tab at top) --> Change runtime type, then select GPU under Hardware accelerator

---

**Next:**  Run all sections of the code.  Use defaults if unsure. 

---

**Then:**  Train or Test

---

**NOTES:**  Click folder on the left side <----- to see files. Then click "up arrow on folder" icon to get full list.

In [ ]:
#@markdown ___
#@markdown ## **I. Install Required Packages**

%cd
!git clone --quiet https://github.com/zhixuhao/unet.git

!pip uninstall -y tensorflow
!pip install tensorflow_gpu==1.14
!pip install keras==2.2.4

In [ ]:
#@markdown ___
#@markdown ## **II. Import Libraries and Connect to Drive**
from google.colab import files
from google.colab import drive
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import skimage

#@markdown Connect to Drive?
DRIVE = True #@param {type : "boolean"}
if DRIVE == True:
  drive.mount('/content/drive')

In [ ]:
%cd
#@markdown ___
#@markdown ## **III. Import Image Data**
#@markdown From local PC to cloud, or from drive folder

#@markdown A. Import from PC or Drive?
IMPORT = 'PC' #@param ["PC","Drive"] {type:"string"}

#@markdown B. Specify Whether you will be Conducting Training or Testing?
ToT = "Train" #@param ["Train","Test"] {type:"string"}
#@markdown  . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  . . . . . . . . . . . . . . . . . . . . . . . . . . . . .  . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

#@markdown Additional Options:

#@markdown -- If uploading data from your PC, specify cloud save paths (default examples shown).

#@markdown -- If using data in Drive, specify path to images

#@markdown A. Path to training images
TRAIN_IMG = '/root/train' #@param {type:'string'}

#@markdown B. Path to training labels
TRAIN_LBL = '/root/label' #@param {type:'string'}

#@markdown C. Path to test images
TEST_IMG = '/root/test' #@param {type:'string'}

if IMPORT == 'PC':

  if ToT == "Train":
    if not os.path.exists(TRAIN_IMG):
      os.makedirs(TRAIN_IMG)
    if not os.path.exists(TRAIN_LBL):
      os.makedirs(TRAIN_LBL)
    print("Select Trainig Images to Upload")
    os.chdir(TRAIN_IMG)
    images = files.upload()
    print("Select Trainig Labels to Upload")
    os.chdir(TRAIN_LBL)
    labels = files.upload()
  elif ToT == "Test":
    if not os.path.exists(TEST_IMG):
      os.makedirs(TEST_IMG)
    print("Select Test Images to Upload")
    os.chdir(TEST_IMG)
    imagesT = files.upload()

elif IMPORT == 'Drive':
  if ToT == "Train":
    if not os.path.exists(TRAIN_IMG):
      os.makedirs(TRAIN_IMG)
    if not os.path.exists(TRAIN_LBL):
      os.makedirs(TRAIN_LBL)
    


In [ ]:
#@markdown ## **IV. Configuration**
#@markdown

#@markdown A. Path to root directory
ROOT_PATH = '/root' #@param {type:'string'}

#@markdown B. Folder in root for training images
TRAIN_IMG = "train" #@param {type:'raw'}

#@markdown C. Path to test images
TEST_IMG = TEST_IMG #@param {type:'raw'}

#@markdown D. Folder in root for training labels
TRAIN_LBL = "label" #@param {type:'raw'}

#@markdown E. Save path for test images
SAVE_PATH = '/root/results' #@param {type:'string'}
if not os.path.exists(os.path.join(ROOT_PATH,SAVE_PATH)):
      os.makedirs(os.path.join(ROOT_PATH,SAVE_PATH))

#@markdown F. Load path for pre-trained weights to test with
WEIGHT_PATH = '/content/drive/My Drive/Colab Notebooks/weights/unet_noise_tissue.hdf5' #@param {type:'string'}

#@markdown G. File name for newly trained weights
SAVE_NAME = '/content/drive/My Drive/Colab Notebooks/weights/unet_new_model.hdf5' #@param {type:'string'}

#@markdown H. Image file extension (e.g. png, tif, jpg)
ext = 'png' #@param ['png','tif','jpg']{type:'string'}

#@markdown I. Image color mode (grayscale, rgb)
COLOR = 'grayscale' #@param ['grayscale','rgb','rgba']{type:'string'}

#@markdown J. Select metric to monitor
METRIC = 'loss' #@param ['loss','val_loss','accuracy','val_accuracy']{type:'string'}

#@markdown K. How often to display training info
VERBOSE = 2 #@param {type:"slider", min:0, max:100, step:1}

#@markdown L. Save best weight or last weight
SAVE_BEST = False #@param {type:"boolean"}

#@markdown M. Number of steps per epoch
STEPS = 10 #@param {type: "integer"}

#@markdown N. Number of epochs
EPOCHS = 10 #@param {type: "integer"}

#@markdown O. Size of Image (MxN; resize)
M = 512 #@param {type: "integer"}
N = 512 #@param {type: "integer"}

#@markdown ___

#@markdown ### Data Augmentation Arguements
#@markdown Augmentations are randomly assigned

#@markdown P. Rotation Range (degrees)
ROT = 45 #@param {type:"slider", min:0, max:90, step:1}
ROT = ROT/100

#@markdown Q. Width Shift Range (fraction of total image width)
WSR = 0.15 #@param {type:"slider", min:0, max:0.5, step:0.05}

#@markdown R. Horizontal Shift Range (fraction of total image height)
HSR = 0.15 #@param {type:"slider", min:0, max:0.5, step:0.05}

#@markdown S. Shear angle in counter-clockwise direction (degrees) 
SHEAR = 0 #@param {type:"slider", min:0, max:90, step:1}
SHEAR = SHEAR/100

#@markdown T. Zoom Scale Range
ZOOM = 0.15 #@param {type:"slider", min:0, max:0.50, step:0.05}

#@markdown U. Horizontal Flip
HF = True #@param {type:"boolean"}

#@markdown V. Vertical Flip
VF = True #@param {type:"boolean"}

#@markdown W. Fill mode
FM = 'nearest' #@param ["constant", "nearest", "reflect","wrap"]{type:"string"}


In [ ]:
%cd ~/unet/
#@markdown ___
#@markdown ## **V. Train Network**
#@markdown Using paths and parameters defined above.
from model import *
from data import *

data_gen_args = dict(rotation_range=ROT,
                    width_shift_range=WSR,
                    height_shift_range=HSR,
                    shear_range=SHEAR,
                    zoom_range=ZOOM,
                    horizontal_flip=HF,
                    vertical_flip=VF,
                    fill_mode=FM)

print(ROOT_PATH)
print(TRAIN_IMG)
print(TRAIN_LBL)
myGene = trainGenerator(1,ROOT_PATH, TRAIN_IMG,TRAIN_LBL,data_gen_args,save_to_dir = None, image_color_mode = COLOR)

model = unet()
model_checkpoint = ModelCheckpoint(SAVE_NAME, monitor=METRIC,verbose=VERBOSE, save_best_only=SAVE_BEST)
model.fit_generator(myGene,steps_per_epoch=STEPS,epochs=EPOCHS,callbacks=[model_checkpoint])

In [ ]:
%cd ~/unet/
#@markdown ___
#@markdown ## **VI. Test Network**
#@markdown Test pretrained network
#@markdown
#@markdown NOTE: If using one of our pretrained models, resize images to 512x512
from model import *
from data import *
from matplotlib import pyplot as plt

model = load_model(WEIGHT_PATH)
test_path = TEST_IMG
save_path = SAVE_PATH
container = np.zeros((M,N,1,1))

def bin_ndarray(ndarray, new_shape, operation='sum'):
    """
    J.F. Sebastian
    Bins an ndarray in all axes based on the target shape, by summing or
        averaging.

    Number of output dimensions must match number of input dimensions and 
        new axes must divide old ones.

    """
    operation = operation.lower()
    if not operation in ['sum', 'mean']:
        raise ValueError("Operation not supported.")
    if ndarray.ndim != len(new_shape):
        raise ValueError("Shape mismatch: {} -> {}".format(ndarray.shape,
                                                           new_shape))
    compression_pairs = [(d, c//d) for d,c in zip(new_shape,
                                                  ndarray.shape)]
    flattened = [l for p in compression_pairs for l in p]
    ndarray = ndarray.reshape(flattened)
    for i in range(len(new_shape)):
        op = getattr(ndarray, operation)
        ndarray = op(-1*(i+1))
    return ndarray

def image_normalized(file_path):

    #img = cv2.imread(file_path,0)
    img = skimage.io.imread(file_path)
    img_shape = img.shape
    image_size = (img_shape[1],img_shape[0])
    img_standard = bin_ndarray(img*1.2, (M,N), operation='mean')
    #img_standard = cv2.resize(img, (512, 512), interpolation=cv2.INTER_CUBIC)
    img_new = img_standard
    img_new = np.asarray([img_new / 255.])
    return img_new,image_size

for name in os.listdir(test_path):
  image_path = os.path.join(test_path,name)
  if os.path.isdir(image_path):
    continue
  img,img_size = image_normalized(image_path)
  img = np.reshape(img,img.shape+(1,))
  results = model.predict(img)
  out = np.zeros(img.shape)
  #out = np.zeros((512,512));
  #print(results.shape)
  #out[results[0,:,:,0] > 0] = 1;
  out = 255*results[0,:,:,0];

  _, ax = plt.subplots(1, figsize=(16, 16))
  ax.axis('off')
  plt.imshow(out.astype('uint8'),clim=(0.0, 1.0))
  plt.show()

  skimage.io.imsave(os.path.join(save_path, ("%s") % (name)), out)
  #cv2.imwrite(os.path.join(save_path, ("%s") % (name)), out)



In [ ]:
#@markdown ## **V. Zip and Download Predictions to Local Drive**
#@markdown If download does not occur, check if browser is blocking.
import shutil

output_filename = 'New' #@param {type: 'string'}
dir_name = SAVE_PATH
shutil.make_archive(output_filename, 'zip', dir_name, verbose=1)

files.download(output_filename + '.zip')